In [40]:
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm

import time
import sys

In [41]:
probowlUrl = "https://www.pro-football-reference.com/years/{}/probowl.htm"
probowlHtmlFile = "proBowl{}.html"
probowlId = "pro_bowl" 

collegeStatsUrl = "https://www.sports-reference.com/cfb/years/{}-passing.html"
collegeStatsHtmlFile = "collegeStats{}.html"
collegeStatsId = "div_passing"

combineUrl = "https://www.pro-football-reference.com/draft/{}-combine.htm"
combineHtmlFile = "combineStats{}.html"
combineId = "div_combine" 

In [42]:
def dots(i):
    return "."*i

def scrapeSite(url, htmlFileName, idName, startYear, endYear):
    dfs = []
    rateLimit = False
    i = 1
    for year in range(startYear, endYear+1):
        url_year = url.format(year)
        data = requests.get(url_year)

        with open(htmlFileName.format(year), "w+") as f:
            f.write(data.text)
        with open(htmlFileName.format(year)) as f:

            html_page = f.read()

        soup = BeautifulSoup(html_page, "html.parser")

        try:
            df = soup.find(id=idName)
            df = pd.read_html(str(df))[0]
            dfs.append(df)

            if i == 4:
                print("")
                i = 1
            print("Collecting Data Vate (ve are not blocked)", dots(i), end='\r')
            i = i + 1

        except ValueError:
            print("Rate Limit: Too many requests ve are blocked")
            rateLimit = True

        os.remove(htmlFileName.format(year))

    return rateLimit, dfs 

def getProbowl(rateLimit, dfs):
    if rateLimit == False:
        #append all pro bowl years together into dataframe
        df = pd.concat(dfs)

        df = df.reset_index(drop=True) # needed so that each row has unique index

        # Drop all other positions except qb
        df.drop(df.loc[df['Pos']!="QB"].index, inplace=True)
        # print("only qb df: ", df)

        #Clean data names and add to dict
        regex = re.compile('[^a-zA-Z\s]')
        playerNamesExctract = df.Player.tolist()
        playerNames = {}
        for name in playerNamesExctract:
            playerNames[regex.sub('', name)] = True

        return playerNames
    return None

# need new column names to differentiate passing and rushing yards
def changeTrainColNames(college_stats_df):
    new_cols = []

    for i in range(len(college_stats_df.columns.values)):
        col = college_stats_df.columns.values[i]
        if i in range(0,5):
            new_cols.append(col[len(col)-1])
        else:
            new_cols.append('_'.join(col))

    college_stats_df.columns = new_cols

# some names have "*" next to them which needs to be removed
def removeStarFromName(df):
    index = 0
    for name in df['Player']:
        if "*" in name:
            newName = name.split("*")[0]
            df['Player'].values[index] = newName
        index += 1

def getCollegeStatsDf(rateLimit, dfs):
    if rateLimit == False:
        df = pd.concat(dfs)
        df = df.reset_index(drop=True) # needed so that each row has unique index
        changeTrainColNames(df)
        removeStarFromName(df)
        return df
    return None

def getCombineDf(rateLimit, dfs):
    if rateLimit == False:
        df = pd.concat(dfs)
        df = df.reset_index(drop=True) # needed so that each row has unique index
        df.drop(index=df.loc[df['Pos']!='QB'].index, inplace=True)
        df = df.drop(columns=['School', 'College', 'Drafted (tm/rnd/yr)'])
        return df
    return None

In [43]:
# for training 2008 to 2022
rateLimit, probowlDfs = scrapeSite(probowlUrl, probowlHtmlFile, probowlId, 2008, 2017)
probowl = getProbowl(rateLimit, probowlDfs)

print("\n")
print("Pro-Bowlers: ")
print(probowl)
print(len(probowl))


Pro-Bowlers: 
{'Peyton Manning': True, 'Drew Brees': True, 'Eli Manning': True, 'Brett Favre': True, 'Jay Cutler': True, 'Kerry Collins': True, 'Kurt Warner': True, 'Philip Rivers': True, 'Tony Romo': True, 'Aaron Rodgers': True, 'Matt Schaub': True, 'David Garrard': True, 'Tom Brady': True, 'Vince Young': True, 'Donovan McNabb': True, 'Matt Ryan': True, 'Michael Vick': True, 'Matt Cassel': True, 'Cam Newton': True, 'Andy Dalton': True, 'Ben Roethlisberger': True, 'Andrew Luck': True, 'Russell Wilson': True, 'Robert Griffin III': True, 'Alex Smith': True, 'Nick Foles ': True, 'Matthew Stafford': True, 'Carson Palmer': True, 'Tyrod Taylor': True, 'Teddy Bridgewater': True, 'Derek Carr': True, 'Jameis Winston': True, 'Dak Prescott': True, 'Kirk Cousins': True, 'Carson Wentz': True, 'Jared Goff': True}
36


In [44]:
# for training 2008 to 2017
trainRateLimit, trainDfsStats = scrapeSite(collegeStatsUrl, collegeStatsHtmlFile, collegeStatsId, 2008, 2017) 
# # for validation 2018 to 2019
# valRateLimit, valDfsStats = scrapeSite(collegeStatsUrl, collegeStatsHtmlFile, collegeStatsId, 2018, 2019) 
# # for testing 2020 to 2021
# testRateLimit, testDfsStats = scrapeSite(collegeStatsUrl, collegeStatsHtmlFile, collegeStatsId, 2020, 2021) 

trainDf = getCollegeStatsDf(trainRateLimit, trainDfsStats)

# valDf = getCollegeStatsDf(valRateLimit, valDfsStats)

# testDf = getCollegeStatsDf(testRateLimit, testDfsStats)

In [45]:
trainDf

,Rk,Player,School,Conf,G,Passing_Cmp,Passing_Att,Passing_Pct,Passing_Yds,Passing_Y/A,Passing_AY/A,Passing_TD,Passing_Int,Passing_Rate,Rushing_Att,Rushing_Yds,Rushing_Avg,Rushing_TD
0,1,David Johnson,Tulsa,CUSA,14,258,400,64.5,4059,10.1,10.4,46,18,178.7,92,186,2.0,3
1,2,Sam Bradford,Oklahoma,Big 12,14,328,483,67.9,4720,9.8,11.1,50,8,180.8,42,47,1.1,5
2,3,Zac Robinson,Oklahoma State,Big 12,13,204,314,65.0,3064,9.8,9.9,25,10,166.8,146,562,3.8,8
3,4,Tim Tebow,Florida,SEC,14,192,298,64.4,2746,9.2,10.6,30,4,172.4,176,673,3.8,12
4,5,Nate Davis,Ball State,MAC,14,258,401,64.3,3591,9.0,9.4,26,8,157.0,66,312,4.7,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1080,95,Peyton Bender,Kansas,Big 12,10,148,273,54.2,1609,5.9,5.0,10,10,108.5,25,-87,-3.5,0
1081,96,Steven Williams,Old Dominion,CUSA,10,147,263,55.9,1528,5.8,4.4,6,11,103.9,92,263,2.9,3
1082,97,Max Bortenschlager,Maryland,Big Ten,10,121,233,51.9,1313,5.6,5.5,10,5,109.1,72,13,0.2,2
1083,98,Hasaan Klugh,Charlotte,CUSA,11,133,279,47.7,1524,5.5,4.1,10,13,96.1,137,532,3.9,9


In [46]:
# for training combine 2009 to 2018
trainLimit, trainDfsCombine = scrapeSite(combineUrl,combineHtmlFile,combineId,2009,2018)
# # for validation combine 2019 to 2020
# valLimit, valDfsCombine = scrapeSite(combineUrl,combineHtmlFile,combineId,2019,2020)
# # for testing combine 2021 to 2022
# testLimit, testDfsCombine = scrapeSite(combineUrl,combineHtmlFile,combineId,2021,2022)

combineTrainDf = getCombineDf(trainLimit, trainDfsCombine)

# combineValDf = getCombineDf(trainLimit, trainDfsCombine)

# combineTestDf = getCombineDf(trainLimit, trainDfsCombine)

In [47]:
combineTrainDf

,Player,Pos,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
16,Jason Boltus,QB,6-3,225,4.82,31.5,26,111,7.00,4.47
17,Rhett Bomar,QB,6-2,225,4.70,NaN,25,106,6.91,4.06
20,Tom Brandstater,QB,6-5,220,4.87,28.5,NaN,106,6.93,4.37
29,Nathan Brown,QB,6-1,219,4.86,30.5,NaN,106,7.43,4.44
47,Hunter Cantwell,QB,6-4,235,5.22,26.0,NaN,99,7.40,4.59
...,...,...,...,...,...,...,...,...,...,...
3261,Josh Rosen,QB,6-4,226,4.92,31.0,NaN,111,7.09,4.28
3263,Mason Rudolph,QB,6-5,235,4.90,26.0,NaN,NaN,NaN,NaN
3277,Nic Shimonek,QB,6-3,220,4.88,28.5,NaN,101,7.28,4.32
3329,Mike White,QB,6-5,224,5.09,27.0,NaN,96,7.40,4.40


In [25]:
# add score column with everything preset to 0
trainDf['Score'] = None
score = 0

for i in tqdm(range(10)):
    # fill score column
    for x, row in  trainDf.iterrows():
        if row['Player'] == 'Player':
            continue
        try:
            score = ((((int(row["Passing_Cmp"])/int(row["Passing_Att"]))*100)) + (int(row["Passing_Yds"])/25) + (int(row["Passing_TD"])*4) + (int(row["Rushing_Yds"])/10) + (float(row["Passing_Rate"])/10.0) + (int(row["Rushing_TD"])*6)) - (int(row["Passing_Int"])*2)
            row['Score'] = score

        except ValueError:
            print("error on this row: \n", row)
            break
    time.sleep(0.5)

100%|██████████| 10/10 [00:07<00:00,  1.38it/s]


In [26]:
duplicateDf = trainDf[trainDf.Player.duplicated(keep=False)].sort_values("Player")

duplicates = {}

for x, row in duplicateDf.iterrows():

    if row['Player'] == 'Player':
        continue

    elif row['Player'] not in duplicates:
        duplicates[row['Player']] = dict(row)

    else:

        if row['Score'] > duplicates[row['Player']]['Score']:

            duplicates[row['Player']] = dict(row)

In [28]:
duplicates["Derek Carr"]

{'Rk': '35',
 'Player': 'Derek Carr',
 'School': 'Fresno State',
 'Conf': 'MWC',
 'G': '13',
 'Passing_Cmp': '454',
 'Passing_Att': '659',
 'Passing_Pct': '68.9',
 'Passing_Yds': '5083',
 'Passing_Y/A': '7.7',
 'Passing_AY/A': '8.7',
 'Passing_TD': '50',
 'Passing_Int': '8',
 'Passing_Rate': '156.3',
 'Rushing_Att': '40',
 'Rushing_Yds': '117',
 'Rushing_Avg': '2.9',
 'Rushing_TD': '2',
 'Score': 495.54226100151743}

In [29]:
bestScoreDuplicates = []
for player in duplicates:
    trainDf.drop(trainDf.loc[trainDf['Player']==player].index, inplace=True)
    bestScoreDuplicates.append(pd.DataFrame([duplicates[player]]))
    # trainDf = trainDf.append(duplicates[player], ignore_index = True)

trainDf = pd.concat(bestScoreDuplicates, ignore_index = True)

In [30]:
trainDf

,Rk,Player,School,Conf,G,Passing_Cmp,Passing_Att,Passing_Pct,Passing_Yds,Passing_Y/A,Passing_AY/A,Passing_TD,Passing_Int,Passing_Rate,Rushing_Att,Rushing_Yds,Rushing_Avg,Rushing_TD,Score
0,4,A.J. McCarron,Alabama,SEC,14,211,314,67.2,2933,9.3,10.8,30,3,175.3,49,4,0.1,1,322.447452
1,1,Aaron Murray,Georgia,SEC,14,249,386,64.5,3893,10.1,10.8,36,10,174.8,59,-68,-1.2,3,372.907772
2,34,Aaron Opelt,Toledo,MAC,9,154,261,59.0,1997,7.7,7.2,16,10,135.8,30,164,5.5,3,230.863831
3,65,Adam Weber,Minnesota,Big Ten,13,255,410,62.2,2761,6.7,6.6,15,8,126.9,127,233,1.8,4,276.625122
4,36,Alex Carder,Western Michigan,MAC,12,330,502,65.7,3873,7.7,7.7,31,14,145.3,128,270,2.1,4,382.187052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,33,Zach Collaros,Cincinnati,Big East,11,225,383,58.7,2902,7.6,7.3,26,14,137.5,121,202,1.7,4,308.776736
301,48,Zach Maynard,California,Pac-12,13,231,405,57.0,2990,7.4,6.9,17,12,127.0,84,108,1.3,4,268.137037
302,3,Zach Mettenberger,LSU,SEC,12,192,296,64.9,3082,10.4,10.7,22,8,171.4,34,-133,-3.9,0,263.984865
303,39,Zach Smith,Baylor,Big 12,10,116,196,59.2,1526,7.8,7.5,13,7,139.3,25,-26,-1.0,1,175.553673


In [31]:
combineTrainDf

,Player,Pos,Ht,Wt,40yd,Vertical,Bench,Broad Jump,3Cone,Shuttle
16,Jason Boltus,QB,6-3,225,4.82,31.5,26,111,7.00,4.47
17,Rhett Bomar,QB,6-2,225,4.70,NaN,25,106,6.91,4.06
20,Tom Brandstater,QB,6-5,220,4.87,28.5,NaN,106,6.93,4.37
29,Nathan Brown,QB,6-1,219,4.86,30.5,NaN,106,7.43,4.44
47,Hunter Cantwell,QB,6-4,235,5.22,26.0,NaN,99,7.40,4.59
...,...,...,...,...,...,...,...,...,...,...
3261,Josh Rosen,QB,6-4,226,4.92,31.0,NaN,111,7.09,4.28
3263,Mason Rudolph,QB,6-5,235,4.90,26.0,NaN,NaN,NaN,NaN
3277,Nic Shimonek,QB,6-3,220,4.88,28.5,NaN,101,7.28,4.32
3329,Mike White,QB,6-5,224,5.09,27.0,NaN,96,7.40,4.40
